# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
print(os.getcwd())
filepath = os.getcwd() + '/event_data'
for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
full_data_rows_list = [] 
for f in file_path_list:
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
        for line in csvreader:
            full_data_rows_list.append(line) 
            
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


#### Creating a Cluster

In [5]:
from cassandra.cluster import Cluster
cluster = Cluster()
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

#### Create table 'songs_by_user_session'. Partition keys: 'sessionId' and 'itemInSession'. The query will use these column as a filter.

In [8]:
query = "CREATE TABLE IF NOT EXISTS songs_by_user_session"
query = query + "(sessionId int, itemSession int, artist_name text, song_title text, length double,  PRIMARY KEY (sessionId, itemSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)                    

#### Insert data into 'songs_by_user_session' table

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO songs_by_user_session (sessionId, itemSession, artist_name, song_title, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Verify that the data have been inserted into 'songs_by_user_session'

In [10]:
query = "select artist_name, song_title, length from songs_by_user_session where sessionId=338 and itemSession=4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist_name, row.song_title, row.length)


Faithless Music Matters (Mark Knight Dub) 495.3073


#### Create table 'songs_by_user_and_session'. Partition keys: 'userId' and 'sessionId'. The query will use these column as a filter. Column: 'itemSession' will be used as a cluster.

In [11]:
query = "CREATE TABLE IF NOT EXISTS songs_by_user_and_session"
query = query + "(userId int, sessionId int, artist_name text, song_title text, itemSession int,  first_name text, last_name text, PRIMARY KEY ((userId, sessionId), itemSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)      
                    

#### Insert data into into 'songs_by_user_and_session'

In [12]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO songs_by_user_and_session (userId, sessionId, artist_name, song_title, itemSession,  first_name, last_name)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), line[0], line[9], int(line[3]), line[1], line[4]))

#### Verify that the data have been inserted into 'songs_by_user_and_session'

In [13]:
query = "select artist_name, song_title, first_name, last_name from songs_by_user_and_session where usedId=10 and sessionId=182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist_name, row.song_title, row.first_name, row.last_name)

Error from server: code=2200 [Invalid query] message="Undefined column name usedid"


#### Create table 'user_by_song'. Partition keys: 'song' and 'userId'. The 'song' column will be used as a filter, and 'userId' will compose a unique key since use IDs are already unique identifiers.

In [14]:
query = "CREATE TABLE IF NOT EXISTS user_by_song"
query = query + "(song text,  userId int, first_name text, last_name text, PRIMARY KEY (song, userId))"
try:
    session.execute(query)
except Exception as e:
    print(e)      

#### Insert data into into 'user_by_song'

In [15]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_by_song (song, userId,first_name, last_name)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]),line[1], line[4]))

#### Verify that the data have been inserted into 'user_by_song'

In [16]:
query = "select first_name, last_name from user_by_song where song='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.first_name, row.last_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [17]:
query = "drop table if exists songs_by_user_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table if exists songs_by_user_and_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table if exists user_by_song"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()